# Is the FIFA World Cup Draw Truly Random?

### Pratik Kulkarni - 19400570
### Shrey Parekh - 18706941
\
***Project for 301111 Discovery Project***\
\
**Supervisor: Dr. Russell Thomson**\
\
***School of Computer, Data and Mathematical Sciences,***\
***Western Sydney University***

**Spring, 2021**
***

In [1]:
#Here we import the libraries we will be using for this project.

import pandas as pd
import numpy as np
import math
import seaborn as sns
import pandas as pd

from scipy import stats
from matplotlib import pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

***

## 1 Fifa Rankings

In [2]:
#Here we load the data set and store in the variable fifaRanks
#We use the pandas library to read the data set
fifaRanks = pd.read_csv("fifa_ranking-2020-12-10.csv")

In [3]:
display(fifaRanks)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,43885,104,Swaziland,SWZ,10,0,0,CAF,1992-12-31
1,43972,42,Turkey,TUR,31,0,0,UEFA,1992-12-31
2,43952,43,Northern Ireland,NIR,31,0,0,UEFA,1992-12-31
3,43945,44,Finland,FIN,31,0,0,UEFA,1992-12-31
4,43976,45,Australia,AUS,29,0,0,AFC,1992-12-31
...,...,...,...,...,...,...,...,...,...
62419,43840,74,United Arab Emirates,UAE,1326,1326,0,AFC,2020-12-10
62420,43812,75,China PR,CHN,1323,1323,0,AFC,2020-12-10
62421,1930407,76,Curaçao,CUW,1313,1313,0,CONCACAF,2020-12-10
62422,43901,51,Costa Rica,CRC,1427,1427,0,CONCACAF,2020-12-10


In [4]:
# convert date column into date format
fifaRanks['rank_date'] = pd.to_datetime(fifaRanks['rank_date'])

# Change old team name of Côte d'Ivoire to Ivory Coast
fifaRanks[['country_full']] = fifaRanks[['country_full']].replace("Côte d'Ivoire", "Ivory Coast")

# filter rows on the basis of date
fifaRanks_filtered = (fifaRanks['rank_date'] > '1994-01-01') & (fifaRanks['rank_date'] <= '2015-1-1')
  
# locate rows and access them using .loc() function
fifaRanks_filtered = fifaRanks.loc[fifaRanks_filtered]
  
# print dataframe
display(fifaRanks_filtered)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
995,43943,109,Estonia,EST,11,11,0,UEFA,1994-02-15
996,43916,165,St. Kitts and Nevis,SKN,0,0,1,CONCACAF,1994-02-15
997,44002,58,Slovakia,SVK,30,2,92,UEFA,1994-02-15
998,43817,57,IR Iran,IRN,30,31,2,AFC,1994-02-15
999,43826,165,Macau,MAC,0,0,1,AFC,1994-02-15
...,...,...,...,...,...,...,...,...,...
48965,44003,135,Tajikistan,TJK,215,215,0,AFC,2014-12-18
48966,43994,135,Azerbaijan,AZE,215,215,0,UEFA,2014-12-18
48967,43978,134,New Zealand,NZL,216,216,1,OFC,2014-12-18
48968,43862,133,Guinea-Bissau,GNB,218,218,2,CAF,2014-12-18


In [5]:
# Get year from rank_date and extract into new dataframe
fifaRanks_years = pd.DataFrame(fifaRanks_filtered['rank_date'].dt.year)

# Rename column name from rank_date to rank_year
fifaRanks_years.columns = ['rank_year']

display(fifaRanks_years)

,rank_year
995,1994
996,1994
997,1994
998,1994
999,1994
...,...
48965,2014
48966,2014
48967,2014
48968,2014


In [6]:
fifaRanks_filtered = fifaRanks_filtered.join(fifaRanks_years)

display(fifaRanks_filtered)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,rank_year
995,43943,109,Estonia,EST,11,11,0,UEFA,1994-02-15,1994
996,43916,165,St. Kitts and Nevis,SKN,0,0,1,CONCACAF,1994-02-15,1994
997,44002,58,Slovakia,SVK,30,2,92,UEFA,1994-02-15,1994
998,43817,57,IR Iran,IRN,30,31,2,AFC,1994-02-15,1994
999,43826,165,Macau,MAC,0,0,1,AFC,1994-02-15,1994
...,...,...,...,...,...,...,...,...,...,...
48965,44003,135,Tajikistan,TJK,215,215,0,AFC,2014-12-18,2014
48966,43994,135,Azerbaijan,AZE,215,215,0,UEFA,2014-12-18,2014
48967,43978,134,New Zealand,NZL,216,216,1,OFC,2014-12-18,2014
48968,43862,133,Guinea-Bissau,GNB,218,218,2,CAF,2014-12-18,2014


## 2 FIFA Groups and Matches

In [7]:
# View Matches dataset
fifa_all = pd.read_csv("WorldCupMatches.csv")

fifa_all.shape

display(fifa_all)

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning Matches Dataset

In [8]:
## Clean unnessary columns
fifa_clean = fifa_all.drop(["Datetime","Stadium", "City", "Attendance", "Referee", "Assistant 1", "Assistant 2", "MatchID"], axis=1)

## Remove trailing NAs
fifa_clean = fifa_clean.dropna(how='all')

# Change old team name of Cote d'Ivoire to Ivory Coast
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace("C�te d'Ivoire", "Ivory Coast")

# Remove extra code chunk of " rn"> "
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace({'rn">':''}, regex=True)

# Filter rows based on years from 1994 to 2014
newdf = (fifa_clean['Year'] >= 1994) & (fifa_clean['Year'] <= 2014)
  
# locate rows and access them using .loc() function
fifa_clean = fifa_clean.loc[newdf]

display(fifa_clean)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials
464,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR
465,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL
466,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI
467,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL
468,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU
...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC
848,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER
849,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG
850,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED


***

## Merging Ranks and Matches Dataset

In [9]:
# Merge cleaned fifa matches dataset and fifaRanks dataset based on team names and year

fifaMatchesRanks = pd.merge(fifa_clean, fifaRanks_filtered,  how='left', left_on=['Home Team Name','Year'], right_on = ['country_full','rank_year'])

# Drop extra columns from fifaRanks dataset
fifaMatchesRanks = fifaMatchesRanks.drop(["id","country_full", "country_abrv", "confederation", "rank_date", "rank_year"], axis=1)

display(fifaMatchesRanks)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
0,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,57.0,57.0,1.0
1,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,56.0,57.0,0.0
2,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,7.0,55.0,56.0,1.0
3,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,9.0,54.0,55.0,2.0
4,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,5.0,56.0,54.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1736.0,1724.0,0.0
4213,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1765.0,1736.0,0.0
4214,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1669.0,1765.0,0.0
4215,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1725.0,1669.0,0.0


***

## Creating dataframes for Years 1994-2014

In [10]:
def extract_fifayear(year):
    
    # only keep rows where Year == 2014
    fifayear = pd.DataFrame(fifaMatchesRanks.loc[fifaMatchesRanks['Year'] == year])
        
    # only keep Group "X" rows to get groups in FIFA 'year' Matches.
    fifayear_groups = pd.DataFrame(fifayear.loc[:(fifayear == 'Round of 16').any(1).idxmax()])
        
    #drop duplicates
    fifayear_groups_unique = pd.DataFrame(fifayear_groups.drop_duplicates(subset=['Home Team Name']))

    # alphabetical sort
    fifayear_groups_unique = pd.DataFrame(fifayear_groups_unique.sort_values('Stage'))

    #remove all extra columns
    fifayear_groups_unique = pd.DataFrame(
        fifayear_groups_unique.drop(fifayear_groups_unique.columns[[range(3, 12, 1)]], axis=1)
)
         
    return fifayear, fifayear_groups, fifayear_groups_unique

In [11]:
fifa2014, fifa2014_groups, fifa2014_groups_unique = extract_fifayear(2014)

display(fifa2014)
display(fifa2014_groups)
display(fifa2014_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
3257,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,10.0,1102.0,1102.0,0.0
3258,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,9.0,1125.0,1102.0,1.0
3259,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,9.0,1104.0,1125.0,0.0
3260,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,6.0,1174.0,1104.0,3.0
3261,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,4.0,1210.0,1174.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1736.0,1724.0,0.0
4213,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1765.0,1736.0,0.0
4214,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1669.0,1765.0,0.0
4215,2014.0,Final,Germany,1.0,0.0,Argentina,Germany win after extra time,0.0,0.0,255959.0,GER,ARG,1.0,1725.0,1669.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
3257,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,10.0,1102.0,1102.0,0.0
3258,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,9.0,1125.0,1102.0,1.0
3259,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,9.0,1104.0,1125.0,0.0
3260,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,6.0,1174.0,1104.0,3.0
3261,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,4.0,1210.0,1174.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829,2014.0,Group H,Algeria,1.0,1.0,Russia,,0.0,1.0,255931.0,ALG,RUS,20.0,926.0,880.0,4.0
3830,2014.0,Group H,Algeria,1.0,1.0,Russia,,0.0,1.0,255931.0,ALG,RUS,15.0,989.0,926.0,5.0
3831,2014.0,Group H,Algeria,1.0,1.0,Russia,,0.0,1.0,255931.0,ALG,RUS,18.0,948.0,989.0,3.0
3832,2014.0,Group H,Algeria,1.0,1.0,Russia,,0.0,1.0,255931.0,ALG,RUS,18.0,948.0,948.0,0.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
3257,2014.0,Group A,Brazil,10.0,1102.0,1102.0,0.0
3269,2014.0,Group A,Mexico,21.0,892.0,892.0,0.0
3677,2014.0,Group A,Croatia,16.0,971.0,971.0,0.0
3485,2014.0,Group A,Cameroon,50.0,616.0,615.0,0.0
3281,2014.0,Group B,Spain,1.0,1507.0,1507.0,0.0
3293,2014.0,Group B,Chile,15.0,1005.0,1014.0,0.0
3653,2014.0,Group B,Netherlands,9.0,1106.0,1106.0,0.0
3461,2014.0,Group B,Australia,56.0,571.0,565.0,2.0
3521,2014.0,Group C,Japan,48.0,641.0,638.0,1.0
3725,2014.0,Group C,Greece,12.0,1055.0,1055.0,0.0


In [12]:
fifa2010, fifa2010_groups, fifa2010_groups_unique = extract_fifayear(2010)

display(fifa2010)
display(fifa2010_groups)
display(fifa2010_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
2553,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,81.0,391.0,377.0,4.0
2554,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,81.0,391.0,391.0,0.0
2555,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,88.0,372.0,391.0,7.0
2556,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,90.0,369.0,372.0,2.0
2557,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,83.0,392.0,369.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,2010.0,Final,Netherlands,0.0,1.0,Spain,Spain win after extra time,0.0,0.0,249721.0,NED,ESP,2.0,1659.0,1659.0,0.0
3253,2010.0,Final,Netherlands,0.0,1.0,Spain,Spain win after extra time,0.0,0.0,249721.0,NED,ESP,2.0,1663.0,1659.0,0.0
3254,2010.0,Final,Netherlands,0.0,1.0,Spain,Spain win after extra time,0.0,0.0,249721.0,NED,ESP,2.0,1683.0,1663.0,0.0
3255,2010.0,Final,Netherlands,0.0,1.0,Spain,Spain win after extra time,0.0,0.0,249721.0,NED,ESP,2.0,1718.0,1683.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
2553,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,81.0,391.0,377.0,4.0
2554,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,81.0,391.0,391.0,0.0
2555,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,88.0,372.0,391.0,7.0
2556,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,90.0,369.0,372.0,2.0
2557,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,83.0,392.0,369.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,2010.0,Group H,Switzerland,0.0,0.0,Honduras,,0.0,0.0,249722.0,SUI,HON,21.0,882.0,940.0,4.0
3078,2010.0,Group H,Switzerland,0.0,0.0,Honduras,,0.0,0.0,249722.0,SUI,HON,23.0,842.0,882.0,2.0
3079,2010.0,Group H,Switzerland,0.0,0.0,Honduras,,0.0,0.0,249722.0,SUI,HON,21.0,852.0,842.0,2.0
3080,2010.0,Group H,Switzerland,0.0,0.0,Honduras,,0.0,0.0,249722.0,SUI,HON,22.0,840.0,852.0,1.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
2553,2010.0,Group A,South Africa,81.0,391.0,377.0,4.0
2564,2010.0,Group A,Uruguay,21.0,909.0,909.0,1.0
2905,2010.0,Group A,Mexico,17.0,947.0,931.0,0.0
2762,2010.0,Group A,France,7.0,1117.0,1117.0,0.0
2575,2010.0,Group B,Korea Republic,49.0,634.0,625.0,3.0
2586,2010.0,Group B,Argentina,8.0,1082.0,1085.0,0.0
2927,2010.0,Group B,Nigeria,15.0,956.0,848.0,7.0
2751,2010.0,Group B,Greece,12.0,1030.0,1016.0,1.0
2597,2010.0,Group C,England,9.0,1076.0,1076.0,0.0
2608,2010.0,Group C,Algeria,31.0,784.0,823.0,5.0


In [13]:
fifa2006, fifa2006_groups, fifa2006_groups_unique = extract_fifayear(2006)

display(fifa2006)
display(fifa2006_groups)
display(fifa2006_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
1864,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,17.0,707.0,708.0,1.0
1865,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,706.0,707.0,2.0
1866,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,22.0,700.0,706.0,3.0
1867,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,699.0,700.0,3.0
1868,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,696.0,699.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,2006.0,Final,Italy,1.0,1.0,France,Italy win on penalties (5 - 3),0.0,0.0,97410600.0,ITA,FRA,2.0,1550.0,1550.0,0.0
2549,2006.0,Final,Italy,1.0,1.0,France,Italy win on penalties (5 - 3),0.0,0.0,97410600.0,ITA,FRA,5.0,1474.0,1550.0,3.0
2550,2006.0,Final,Italy,1.0,1.0,France,Italy win on penalties (5 - 3),0.0,0.0,97410600.0,ITA,FRA,2.0,1540.0,1474.0,3.0
2551,2006.0,Final,Italy,1.0,1.0,France,Italy win on penalties (5 - 3),0.0,0.0,97410600.0,ITA,FRA,2.0,1560.0,1540.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
1864,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,17.0,707.0,708.0,1.0
1865,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,706.0,707.0,2.0
1866,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,22.0,700.0,706.0,3.0
1867,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,699.0,700.0,3.0
1868,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,19.0,696.0,699.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,2006.0,Group G,Switzerland,2.0,0.0,Korea Republic,,1.0,0.0,97410100.0,SUI,KOR,14.0,1016.0,1028.0,1.0
2374,2006.0,Group G,Switzerland,2.0,0.0,Korea Republic,,1.0,0.0,97410100.0,SUI,KOR,15.0,978.0,1016.0,1.0
2375,2006.0,Group G,Switzerland,2.0,0.0,Korea Republic,,1.0,0.0,97410100.0,SUI,KOR,17.0,937.0,978.0,2.0
2376,2006.0,Group G,Switzerland,2.0,0.0,Korea Republic,,1.0,0.0,97410100.0,SUI,KOR,17.0,937.0,937.0,0.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
1864,2006.0,Group A,Germany,17.0,707.0,708.0,1.0
2046,2006.0,Group A,Ecuador,38.0,647.0,651.0,1.0
2222,2006.0,Group A,Costa Rica,21.0,700.0,699.0,0.0
1875,2006.0,Group A,Poland,22.0,695.0,696.0,0.0
1897,2006.0,Group B,Trinidad and Tobago,50.0,608.0,608.0,0.0
2068,2006.0,Group B,Sweden,14.0,731.0,732.0,0.0
1886,2006.0,Group B,England,9.0,757.0,757.0,0.0
2244,2006.0,Group B,Paraguay,30.0,672.0,672.0,0.0
2278,2006.0,Group C,Ivory Coast,42.0,626.0,626.0,0.0
1919,2006.0,Group C,Serbia and Montenegro,47.0,612.0,612.0,0.0


In [14]:
fifa2002, fifa2002_groups, fifa2002_groups_unique = extract_fifayear(2002)

display(fifa2002)
display(fifa2002_groups)
display(fifa2002_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
1160,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,812.0,812.0,0.0
1161,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,809.0,812.0,0.0
1162,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,807.0,809.0,0.0
1163,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,810.0,807.0,0.0
1164,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,802.0,810.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,2002.0,Final,Germany,0.0,2.0,Brazil,,0.0,0.0,43950600.0,GER,BRA,5.0,764.0,765.0,0.0
1860,2002.0,Final,Germany,0.0,2.0,Brazil,,0.0,0.0,43950600.0,GER,BRA,4.0,770.0,764.0,1.0
1861,2002.0,Final,Germany,0.0,2.0,Brazil,,0.0,0.0,43950600.0,GER,BRA,4.0,763.0,770.0,0.0
1862,2002.0,Final,Germany,0.0,2.0,Brazil,,0.0,0.0,43950600.0,GER,BRA,4.0,763.0,763.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
1160,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,812.0,812.0,0.0
1161,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,809.0,812.0,0.0
1162,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,807.0,809.0,0.0
1163,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,810.0,807.0,0.0
1164,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,1.0,802.0,810.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,2002.0,Group H,Belgium,3.0,2.0,Russia,,1.0,0.0,43950100.0,BEL,RUS,19.0,672.0,677.0,1.0
1685,2002.0,Group H,Belgium,3.0,2.0,Russia,,1.0,0.0,43950100.0,BEL,RUS,17.0,688.0,672.0,2.0
1686,2002.0,Group H,Belgium,3.0,2.0,Russia,,1.0,0.0,43950100.0,BEL,RUS,18.0,682.0,688.0,1.0
1687,2002.0,Group H,Belgium,3.0,2.0,Russia,,1.0,0.0,43950100.0,BEL,RUS,17.0,682.0,682.0,1.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
1160,2002.0,Group A,France,1.0,812.0,812.0,0.0
1171,2002.0,Group A,Uruguay,22.0,664.0,664.0,0.0
1523,2002.0,Group A,Senegal,68.0,536.0,539.0,3.0
1347,2002.0,Group A,Denmark,17.0,672.0,672.0,1.0
1578,2002.0,Group B,Slovenia,27.0,650.0,650.0,2.0
1215,2002.0,Group B,Paraguay,13.0,691.0,691.0,0.0
1237,2002.0,Group B,Spain,7.0,730.0,730.0,0.0
1413,2002.0,Group B,South Africa,33.0,629.0,626.0,2.0
1611,2002.0,Group C,Turkey,23.0,663.0,663.0,0.0
1248,2002.0,Group C,Brazil,3.0,793.0,793.0,0.0


In [15]:
fifa1998, fifa1998_groups, fifa1998_groups_unique = extract_fifayear(1998)

display(fifa1998)
display(fifa1998_groups)
display(fifa1998_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
520,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,73.0,73.0,0.0
521,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,73.0,0.0
522,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,72.0,0.0
523,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,72.0,0.0
524,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,74.0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,1998.0,Final,Brazil,0.0,3.0,France,,0.0,2.0,1027.0,BRA,FRA,1.0,74.0,74.0,0.0
1156,1998.0,Final,Brazil,0.0,3.0,France,,0.0,2.0,1027.0,BRA,FRA,1.0,74.0,74.0,0.0
1157,1998.0,Final,Brazil,0.0,3.0,France,,0.0,2.0,1027.0,BRA,FRA,1.0,74.0,74.0,0.0
1158,1998.0,Final,Brazil,0.0,3.0,France,,0.0,2.0,1027.0,BRA,FRA,1.0,74.0,74.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
520,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,73.0,73.0,0.0
521,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,73.0,0.0
522,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,72.0,0.0
523,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,72.0,72.0,0.0
524,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,74.0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1998.0,Group G,Colombia,0.0,2.0,England,,0.0,2.0,1014.0,COL,ENG,24.0,54.0,57.0,5.0
997,1998.0,Group G,Colombia,0.0,2.0,England,,0.0,2.0,1014.0,COL,ENG,27.0,53.0,54.0,3.0
998,1998.0,Group G,Colombia,0.0,2.0,England,,0.0,2.0,1014.0,COL,ENG,32.0,52.0,53.0,5.0
999,1998.0,Group G,Colombia,0.0,2.0,England,,0.0,2.0,1014.0,COL,ENG,34.0,52.0,52.0,2.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
520,1998.0,Group A,Brazil,1.0,73.0,73.0,0.0
530,1998.0,Group A,Morocco,13.0,59.0,58.0,2.0
680,1998.0,Group A,Scotland,37.0,52.0,52.0,0.0
540,1998.0,Group B,Italy,14.0,58.0,60.0,5.0
550,1998.0,Group B,Cameroon,49.0,47.0,45.0,4.0
700,1998.0,Group B,Chile,7.0,59.0,58.0,9.0
570,1998.0,Group C,Saudi Arabia,32.0,53.0,53.0,1.0
580,1998.0,Group C,France,6.0,59.0,60.0,0.0
720,1998.0,Group C,South Africa,36.0,52.0,53.0,5.0
560,1998.0,Group D,Paraguay,30.0,53.0,54.0,1.0


In [16]:
fifa1994, fifa1994_groups, fifa1994_groups_unique = extract_fifayear(1994)

display(fifa1994)
display(fifa1994_groups)
display(fifa1994_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
0,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,57.0,57.0,1.0
1,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,56.0,57.0,0.0
2,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,7.0,55.0,56.0,1.0
3,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,9.0,54.0,55.0,2.0
4,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,5.0,56.0,54.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,1994.0,Final,Brazil,0.0,0.0,Italy,Brazil win on penalties (3 - 2),0.0,0.0,3459.0,BRA,ITA,1.0,65.0,59.0,2.0
516,1994.0,Final,Brazil,0.0,0.0,Italy,Brazil win on penalties (3 - 2),0.0,0.0,3459.0,BRA,ITA,1.0,66.0,65.0,0.0
517,1994.0,Final,Brazil,0.0,0.0,Italy,Brazil win on penalties (3 - 2),0.0,0.0,3459.0,BRA,ITA,1.0,66.0,66.0,0.0
518,1994.0,Final,Brazil,0.0,0.0,Italy,Brazil win on penalties (3 - 2),0.0,0.0,3459.0,BRA,ITA,1.0,67.0,66.0,0.0


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
0,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,57.0,57.0,1.0
1,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,6.0,56.0,57.0,0.0
2,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,7.0,55.0,56.0,1.0
3,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,9.0,54.0,55.0,2.0
4,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,5.0,56.0,54.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,1994.0,Group D,Argentina,0.0,2.0,Bulgaria,,0.0,0.0,337.0,ARG,BUL,9.0,57.0,57.0,0.0
357,1994.0,Group D,Argentina,0.0,2.0,Bulgaria,,0.0,0.0,337.0,ARG,BUL,9.0,57.0,57.0,0.0
358,1994.0,Group D,Argentina,0.0,2.0,Bulgaria,,0.0,0.0,337.0,ARG,BUL,9.0,58.0,57.0,0.0
359,1994.0,Group D,Argentina,0.0,2.0,Bulgaria,,0.0,0.0,337.0,ARG,BUL,10.0,57.0,58.0,1.0


,Year,Stage,Home Team Name,rank,total_points,previous_points,rank_change
20,1994.0,Group A,USA,21.0,47.0,46.0,1.0
40,1994.0,Group A,Colombia,20.0,49.0,48.0,1.0
250,1994.0,Group A,Switzerland,8.0,56.0,54.0,4.0
130,1994.0,Group A,Romania,18.0,50.0,52.0,5.0
310,1994.0,Group B,Russia,13.0,53.0,52.0,1.0
70,1994.0,Group B,Cameroon,29.0,44.0,46.0,6.0
90,1994.0,Group B,Brazil,2.0,58.0,59.0,1.0
180,1994.0,Group B,Sweden,10.0,54.0,55.0,1.0
0,1994.0,Group C,Spain,6.0,57.0,57.0,1.0
10,1994.0,Group C,Germany,1.0,59.0,59.0,0.0


***

***

## World Cup Results Dataset

In [17]:
fifaResults = pd.read_csv("WorldCups.csv")

fifaResults.shape

display(fifaResults)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819.810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893.172
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1.563.135
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1.603.975
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1.865.753


In [18]:
fifaResults_filtered = (fifaResults['Year'] >= 1994)

# locate rows and access them using .loc() function
fifaResults_filtered = fifaResults.loc[fifaResults_filtered]

display(fifaResults_filtered)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
14,1994,USA,Brazil,Italy,Sweden,Bulgaria,141,24,52,3.587.538
15,1998,France,France,Brazil,Croatia,Netherlands,171,32,64,2.785.100
16,2002,Korea/Japan,Brazil,Germany,Turkey,Korea Republic,161,32,64,2.705.197
17,2006,Germany,Italy,France,Germany,Portugal,147,32,64,3.359.439
18,2010,South Africa,Spain,Netherlands,Germany,Uruguay,145,32,64,3.178.856
19,2014,Brazil,Germany,Argentina,Netherlands,Brazil,171,32,64,3.386.810


***